<a href="https://colab.research.google.com/github/virtualacademy-pk/python/blob/main/Partial_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Partial Functions

In [ ]:
from functools import partial

In [ ]:
def my_func(a, b, c):
    print(a, b, c)

In [ ]:
f = partial(my_func, 10)

In [ ]:
f(20, 30)

10 20 30


We could have done this using another function (or a lambda) as well:

In [ ]:
def partial_func(b, c):
    return my_func(10, b, c)

In [ ]:
partial_func(20, 30)

10 20 30


or, using a lambda:

In [ ]:
fn = lambda b, c: my_func(10, b, c)

In [ ]:
fn(20, 30)

10 20 30


Any of these ways is fine, but sometimes partial is just a cleaner more consise way to do it.

Also, it is quite flexible with parameters:

In [ ]:
def my_func(a, b, *args, k1, k2, **kwargs):
    print(a, b, args, k1, k2, kwargs)

In [ ]:
f = partial(my_func, 10, k1='a')

In [ ]:
f(20, 30, 40, k2='b', k3='c')

10 20 (30, 40) a b {'k3': 'c'}


We can of course do the same thing using a regular function too:

In [ ]:
def f(b, *args, k2, **kwargs):
    return my_func(10, b, *args, k1='a', k2=k2, **kwargs)

In [ ]:
f(20, 30, 40, k2='b', k3='c')

10 20 (30, 40) a b {'k3': 'c'}


As you can see in this case, using **partial** seems a lot simpler.

Also, you are not stuck having to specify the first argument in your partial:

In [ ]:
def power(base, exponent):
    return base ** exponent

In [ ]:
power(2, 3)

8

In [ ]:
square = partial(power, exponent=2)

In [ ]:
square(4)

16

In [ ]:
cube = partial(power, exponent=3)

In [ ]:
cube(2)

8

You can even call it this way:

In [ ]:
cube(base=3)

27

#### Caveat

We can certainly use variables instead of literals when creating partials, but we have to be careful.

In [ ]:
def my_func(a, b, c):
    print(a, b, c)

In [ ]:
a = 10
f = partial(my_func, a)

In [ ]:
f(20, 30)

10 20 30


Now let's change the value of the variable **a** and see what happens:

In [ ]:
a = 100

In [ ]:
f(20, 30)

10 20 30


As you can see, the value for **a** is fixed once the partial has been created.

In fact, the memory address of **a** is baked in to the partial, and **a** is immutable.

If we use a mutable object, things are different:

In [ ]:
a = [10, 20]
f = partial(my_func, a)

In [ ]:
f(100, 200)

[10, 20] 100 200


In [ ]:
a.append(30)

In [ ]:
f(100, 200)

[10, 20, 30] 100 200


#### Use Cases

We tend to use partials in situation where we need to call a function that actually requires more parameters than we can supply.

Often this is because we are working with exiting libraries or code, and we have a special case.

For example, suppose we have points (represented as tuples), and we want to sort them based on the distance of the point from some other fixed point:

In [ ]:
origin = (0, 0)

In [ ]:
l = [(1,1), (0, 2), (-3, 2), (0,0), (10, 10)]

In [ ]:
dist2 = lambda x, y: (x[0]-y[0])**2 + (x[1]-y[1])**2

In [ ]:
dist2((0,0), (1,1))

2

In [ ]:
sorted(l, key = lambda x: dist2((0,0), x))

[(0, 0), (1, 1), (0, 2), (-3, 2), (10, 10)]

In [ ]:
sorted(l, key=partial(dist2, (0,0)))

[(0, 0), (1, 1), (0, 2), (-3, 2), (10, 10)]

Another use case is when using **callback** functions. Usually these are used when running asynchronous operations, and you provide a callable to another callable which will be called when the first callable completes its execution.

Very often, the asynchronous callable will specify the number of variables that the callback function must have - this may not be what we want, maybe we want to add some additional info.

We'll look at asynchronous processing later in this course.

Often we can also use partial functions to make our life a bit easier.

Consider a situation where we have some generic `email()` function that can be used to notify someone when various things happen in our application. But depending on what is happening we may want to notify different people. Let's see how we may do this:

In [ ]:
def sendmail(to, subject, body):
    # code to send email
    print('To:{0}, Subject:{1}, Body:{2}'.format(to, subject, body))

Now, we may haver different email adresses we want to send notifications to, maybe defined in a config file in our app. Here, I'll just use hardcoded variables:

In [ ]:
email_admin = 'palin@python.edu'
email_devteam = 'idle@python.edu;cleese@python.edu'

Now when we want to send emails we would have to write things like:

In [ ]:
sendmail(email_admin, 'My App Notification', 'the parrot is dead.')
sendmail(';'.join((email_admin, email_devteam)), 'My App Notification', 'the ministry is closed until further notice.')

To:palin@python.edu, Subject:My App Notification, Body:the parrot is dead.
To:palin@python.edu;idle@python.edu;cleese@python.edu, Subject:My App Notification, Body:the ministry is closed until further notice.


We could simply our life a little using partials this way:

In [ ]:
send_admin = partial(sendmail, email_admin, 'For you eyes only')
send_dev = partial(sendmail, email_devteam, 'Dear IT:')
send_all = partial(sendmail, ';'.join((email_admin, email_devteam)), 'Loyal Subjects')

In [ ]:
send_admin('the parrot is dead.')
send_all('the ministry is closed until further notice.')

To:palin@python.edu, Subject:For you eyes only, Body:the parrot is dead.
To:palin@python.edu;idle@python.edu;cleese@python.edu, Subject:Loyal Subjects, Body:the ministry is closed until further notice.


Finally, let's make this a little more complex, with a mixture of positional and keyword-only arguments:

In [ ]:
def sendmail(to, subject, body, *, cc=None, bcc=email_devteam):
    # code to send email
    print('To:{0}, Subject:{1}, Body:{2}, CC:{3}, BCC:{4}'.format(to, 
                                                                  subject, 
                                                                  body, 
                                                                  cc, 
                                                                  bcc))

In [ ]:
send_admin = partial(sendmail, email_admin, 'General Admin')
send_admin_secret = partial(sendmail, email_admin, 'For your eyes only', cc=None, bcc=None)

In [ ]:
send_admin('and now for something completely different')

To:palin@python.edu, Subject:General Admin, Body:and now for something completely different, CC:None, BCC:idle@python.edu;cleese@python.edu


In [ ]:
send_admin_secret('the parrot is dead!')

To:palin@python.edu, Subject:For your eyes only, Body:the parrot is dead!, CC:None, BCC:None


In [ ]:
send_admin_secret('the parrot is no more!', bcc=email_devteam)

To:palin@python.edu, Subject:For your eyes only, Body:the parrot is no more!, CC:None, BCC:idle@python.edu;cleese@python.edu


In [ ]:
def pow(base, exponent):
    return base ** exponent

In [ ]:
cube = partial(pow, exponent=3)

In [ ]:
cube(2)

8

In [ ]:
cube(2, 4)

TypeError: pow() got multiple values for argument 'exponent'

In [ ]:
cube(2, exponent=4)

16